In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Hardika\Downloads\archive (2)\Training Dataset.csv")
docs = df.apply(lambda row: row.to_json(), axis=1).tolist()


In [2]:
df.head()
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Example docs
docs = ["Applicant has good income", "Loan amount is high", "Credit history is clear"]

# Function to embed text
def embed(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        # Mean pooling over tokens
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()

# Embed all docs
embeddings = np.vstack([embed(doc) for doc in docs])

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print("FAISS index built ")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\Hardika\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hardika\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

FAISS index built 


In [7]:
# Example user query
query = "Who will get the loan approved?"

# Embed the query
query_emb = embed(query).reshape(1, -1)

# Search FAISS index for top 3 matches
D, I = index.search(query_emb, k=3)

# Print best matching docs
print("Query:", query)
print("Top matches:")
for idx in I[0]:
    print("-", docs[idx])


Query: Who will get the loan approved?
Top matches:
- Credit history is clear
- Loan amount is high
- Applicant has good income


In [14]:
from transformers import pipeline

# Force PyTorch to avoid TensorFlow/Keras issue
generator = pipeline("text2text-generation", model="google/flan-t5-small", framework="pt")

# Combine retrieved docs as context
retrieved_docs = [docs[idx] for idx in I[0]]
context = "\n".join(retrieved_docs)

# Build prompt
prompt = f"""Context:
{context}

Question: {query}

Answer:"""

# Generate
result = generator(prompt, max_length=128)
print("Generated Answer:\n", result[0]['generated_text'])



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)a5b18a05535c9e14c7a355904270e15b0945ea86:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Answer:
 the applicant


In [15]:
# Try more user queries
new_queries = [
    "What factors affect loan approval?",
    "Is high income important?",
    "What if credit history is bad?"
]

for q in new_queries:
    # Embed query
    q_emb = embed(q).reshape(1, -1)

    # Search top docs
    D, I = index.search(q_emb, k=3)
    retrieved_docs = [docs[idx] for idx in I[0]]
    context = "\n".join(retrieved_docs)

    prompt = f"""Context:
{context}

Question: {q}

Answer:"""

    result = generator(prompt, max_length=128)
    print(f"\nQuery: {q}")
    print("Generated Answer:", result[0]['generated_text'])


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Query: What factors affect loan approval?
Generated Answer: credit history is clear Loan amount is high Applicant has good income


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Query: Is high income important?
Generated Answer: Loan amount is high

Query: What if credit history is bad?
Generated Answer: i.


In [16]:
#Interactive Q and A
while True:
    user_query = input("\nAsk a question (or type 'exit'): ")
    if user_query.lower() == "exit":
        break

    q_emb = embed(user_query).reshape(1, -1)
    D, I = index.search(q_emb, k=3)
    retrieved_docs = [docs[idx] for idx in I[0]]
    context = "\n".join(retrieved_docs)

    prompt = f"""Context:
{context}

Question: {user_query}

Answer:"""

    result = generator(prompt, max_length=128)
    print("Answer:", result[0]['generated_text'])



Ask a question (or type 'exit'):  Is it good if the applicant has a clear credit history?


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: yes



Ask a question (or type 'exit'):  What other factors decide loan approval?


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Answer: yes



Ask a question (or type 'exit'):  What if the applicant has bad credit?


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: credit history is clear Loan amount is high



Ask a question (or type 'exit'):  exit
